XXX 
## Things that need to be done 
1) For the multi class classifier - it takes a very long time - check if there is sthg to do about it
2) Metrics for river for classifiers (F1 and accuracy) are almost all null - fix this.
3) Do all the comparisons
4) Comment these results
5) The whole part about the anomaly detection

In [10]:
# General
import pandas as pd
import time

# CapyMOA 
from capymoa import datasets
from capymoa.classifier import AdaptiveRandomForestClassifier, HoeffdingAdaptiveTree, HoeffdingTree, LeveragingBagging, KNN, NaiveBayes, OnlineAdwinBagging, OnlineBagging, SGDClassifier
from capymoa.regressor import PassiveAggressiveRegressor, SGDRegressor, TargetMean, KNNRegressor, AdaptiveRandomForestRegressor
from capymoa.evaluation import prequential_evaluation

# River
from river import stream
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import naive_bayes
from river import tree
from river import forest
from river import dummy
from river import stats
from river import neighbors

## Part I - Binary Classifiers

The classifiers I found on both River and CapyMOA were : 
- ADWIN Bagging, (capymoa OnlineAdwinBagging)
- Adaptive Random Forest,
- Bagging, 
- Hoeffding Adaptive Tree, 
- Hoeffding Tree, 
- Leveraging Bagging, 
- Naive Bayes, 
- k-Nearest Neighbors, 
- sklearn SGDClassifier

We will compare both libraries on these classifiers.

The datasets we will use are from CapyMOA and are the following : 
- Electricity
- Electricity tiny

### I.1) Getting the data from CapyMOA 

In [12]:
elec_stream = datasets.Electricity()
variables = ['1', '2', '3', '4', '5', '6', '7','8' ] #XXX - get the real variables

def dataset_to_csv(stream, file_name, variables):
    data = []
    while stream.has_more_instances():
        inst = {}
        instance = stream.next_instance()
        x = instance.x
        try : 
            y = instance.y_index
        except:
            y = instance.y_value
            
        for i in range (len(x)) : 
            inst[variables[i]] = x[i]
        inst['target'] = y
        data.append(inst)
    
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False, sep = ',')

dataset_to_csv(elec_stream, 'elec_stream.csv', variables)


In [13]:
elec_stream_tiny = datasets.ElectricityTiny()
variables = [str(i) for i in range (1, 7)] #XXX - get the real variables
dataset_to_csv(elec_stream_tiny, 'elec_stream_tiny.csv', variables)

Hyper100k = datasets.Hyper100k()
variables = [str(i) for i in range (1, 11)] #XXX - get the real variables
dataset_to_csv(Hyper100k, 'Hyper100k.csv', variables)



## I.2) Evaluating with CapyMOA

In [15]:
Datasets_capymoa = {'Electricity' : elec_stream, 'Electricity Tiny': elec_stream_tiny, 'Hyper100k': Hyper100k}
Models_capymoa = {'ADWIN bagging': OnlineAdwinBagging,
         'Adaptive Random Forest': AdaptiveRandomForestClassifier ,
         'Bagging': OnlineBagging,
         'Hoeffding Adaptive Tree': HoeffdingAdaptiveTree ,
         'Hoeffding Tree': HoeffdingTree,
         'Leveraging Bagging': LeveragingBagging,
         'Naive Bayes': NaiveBayes,
         'k-Nearest Neighbors': KNN,
         'SGDClassifier': SGDClassifier}
#metrics_river = {'F1' : metrics.F1(), 'Accuracy' : metrics.Accuracy()}

def get_performance_capymoa(Models, Datasets):
    results = []
    for model_name in Models.keys():
        model = Models[model_name]
        for data_name in Datasets.keys():
            stream = Datasets[data_name]
            print(f"CapyMOA : Model {model_name} and dataset {data_name}")
            result_model = dict()
            beginning = time.time()
            
            classifier = model(stream.get_schema())
            res = prequential_evaluation(stream, classifier)
            end = time.time()
            try :
                result_model = {'Model': model_name,
                    'Dataset': data_name,
                    'Time (seconds)': round(end - beginning, 4),
                    'Accuracy': res['cumulative'].accuracy(),
                    'F1': res['cumulative'].f1_score()}
            except:
                result_model = {'Model': model_name,
                    'Dataset': data_name,
                    'Time (seconds)': round(end - beginning, 4),
                    'RMSE': res['cumulative'].rmse(),
                    'MAE': res['cumulative'].mae(),
                    'R2': res['cumulative'].r2()}
            results.append(result_model)
    return results



In [16]:
results_capymoa = get_performance_capymoa(Models_capymoa, Datasets_capymoa)

CapyMOA : Model ADWIN bagging and dataset Electricity
CapyMOA : Model ADWIN bagging and dataset Electricity Tiny
CapyMOA : Model ADWIN bagging and dataset Hyper100k
CapyMOA : Model Adaptive Random Forest and dataset Electricity
CapyMOA : Model Adaptive Random Forest and dataset Electricity Tiny
CapyMOA : Model Adaptive Random Forest and dataset Hyper100k
CapyMOA : Model Bagging and dataset Electricity
CapyMOA : Model Bagging and dataset Electricity Tiny
CapyMOA : Model Bagging and dataset Hyper100k
CapyMOA : Model Hoeffding Adaptive Tree and dataset Electricity
CapyMOA : Model Hoeffding Adaptive Tree and dataset Electricity Tiny
CapyMOA : Model Hoeffding Adaptive Tree and dataset Hyper100k
CapyMOA : Model Hoeffding Tree and dataset Electricity
CapyMOA : Model Hoeffding Tree and dataset Electricity Tiny
CapyMOA : Model Hoeffding Tree and dataset Hyper100k
CapyMOA : Model Leveraging Bagging and dataset Electricity
CapyMOA : Model Leveraging Bagging and dataset Electricity Tiny
CapyMOA : 

In [17]:
pd.DataFrame(results_capymoa)

,Model,Dataset,Time (seconds),Accuracy,F1
0,ADWIN bagging,Electricity,7.4677,84.939972,84.513691
1,ADWIN bagging,Electricity Tiny,0.3208,85.250000,84.821727
2,ADWIN bagging,Hyper100k,20.8275,89.915000,89.914998
3,Adaptive Random Forest,Electricity,45.7416,90.066649,89.807241
4,Adaptive Random Forest,Electricity Tiny,1.3330,89.000000,88.643355
5,Adaptive Random Forest,Hyper100k,139.6235,88.025000,88.034844
6,Bagging,Electricity,5.8263,82.825742,82.348786
7,Bagging,Electricity Tiny,0.3452,85.000000,84.568468
8,Bagging,Hyper100k,17.5200,89.838000,89.838020
9,Hoeffding Adaptive Tree,Electricity,0.2921,83.907133,83.458512


XXX - Comment

### I.3) Evaluating with River

In [19]:
adwin = ensemble.ADWINBaggingClassifier(model=(preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,  seed=42)
bagging = ensemble.BaggingClassifier( model=( preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,seed=42)
lev_bagging = ensemble.LeveragingBaggingClassifier( model=( preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,seed=42)
arf = forest.ARFClassifier(seed = 42)
knn = (preprocessing.StandardScaler() | ensemble.BaggingClassifier( model=( preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,seed=42))
hoeffdingadaptive = preprocessing.StandardScaler() | tree.HoeffdingAdaptiveTreeClassifier(seed=42)
hoeffding = preprocessing.StandardScaler() | tree.HoeffdingTreeClassifier()
SGD = (preprocessing.StandardScaler() | linear_model.LogisticRegression(optim.SGD(0.1)))
NB = preprocessing.StandardScaler() | naive_bayes.GaussianNB()


In [20]:
Datasets_river = {'Electricity' : ('elec_stream.csv', {'converters': {str(i) : float for i in range (1, 9)}}),
                 'Electricity Tiny' : ('elec_stream_tiny.csv', {'converters': {str(i) : float for i in range (1, 7)}}),
                 'Hyper100k' : ('Hyper100k.csv', {'converters': {str(i) : float for i in range (1, 11)}})
                 }

Models_river = {'ADWIN bagging': adwin,
         'Adaptive Random Forest': arf,
         'Bagging': bagging,
         'Hoeffding Adaptive Tree': hoeffdingadaptive,
         'Hoeffding Tree': hoeffding,
         'Leveraging Bagging': lev_bagging,
         'Naive Bayes': NB,
         'k-Nearest Neighbors': knn,
         'SGDClassifier': SGD}

Metrics_river = {'F1' : metrics.F1(), 'Accuracy' : metrics.Accuracy()}

def get_performance_river(Models, Datasets, Metric):
    results = []
    for model_name in Models.keys():
        model = Models[model_name]
        for data_name in Datasets.keys():
            metrics = {name: metr.clone() for name, metr in Metric.items()}
            result_model={}
            beginning = time.time()   #XXX - put the time measurements at the same place
            print(f"River : Model {model_name} and dataset {data_name}")
            dataset_csv, params = Datasets[data_name]
            
            for x, y in stream.iter_csv(dataset_csv, target = 'target', **params):
                model.learn_one(x, y)
                yp = model.predict_one(x)
                for metric_name in Metric.keys():
                    metr = metrics[metric_name]
                    metr.update(y, yp)
            end = time.time()
            
            for metric_name in metrics.keys():
                metr = metrics[metric_name]
                if 'Model' not in result_model.keys():
                    result_model = {'Model': model_name,
                                    'Dataset': data_name,
                                    'Time (seconds)': round(end - beginning, 4),
                                    metric_name: metr}
                else :
                    result_model[metric_name] = metr
            results.append(result_model)
    return results


In [21]:
results_river = get_performance_river(Models_river, Datasets_river, Metrics_river)


River : Model ADWIN bagging and dataset Electricity
River : Model ADWIN bagging and dataset Electricity Tiny
River : Model ADWIN bagging and dataset Hyper100k
River : Model Adaptive Random Forest and dataset Electricity
River : Model Adaptive Random Forest and dataset Electricity Tiny
River : Model Adaptive Random Forest and dataset Hyper100k
River : Model Bagging and dataset Electricity
River : Model Bagging and dataset Electricity Tiny
River : Model Bagging and dataset Hyper100k
River : Model Hoeffding Adaptive Tree and dataset Electricity
River : Model Hoeffding Adaptive Tree and dataset Electricity Tiny
River : Model Hoeffding Adaptive Tree and dataset Hyper100k
River : Model Hoeffding Tree and dataset Electricity
River : Model Hoeffding Tree and dataset Electricity Tiny
River : Model Hoeffding Tree and dataset Hyper100k
River : Model Leveraging Bagging and dataset Electricity
River : Model Leveraging Bagging and dataset Electricity Tiny
River : Model Leveraging Bagging and dataset

In [22]:
pd.DataFrame(results_river)

,Model,Dataset,Time (seconds),F1,Accuracy
0,ADWIN bagging,Electricity,3.8179,F1: 0.00%,Accuracy: 0.00%
1,ADWIN bagging,Electricity Tiny,0.1581,F1: 0.00%,Accuracy: 0.00%
2,ADWIN bagging,Hyper100k,9.5506,F1: 0.00%,Accuracy: 0.00%
3,Adaptive Random Forest,Electricity,26.5067,F1: 0.00%,Accuracy: 95.48%
4,Adaptive Random Forest,Electricity Tiny,1.0280,F1: 0.00%,Accuracy: 94.95%
5,Adaptive Random Forest,Hyper100k,83.5227,F1: 0.00%,Accuracy: 94.52%
6,Bagging,Electricity,2.7262,F1: 0.00%,Accuracy: 0.00%
7,Bagging,Electricity Tiny,0.1070,F1: 0.00%,Accuracy: 0.00%
8,Bagging,Hyper100k,6.9062,F1: 0.00%,Accuracy: 0.00%
9,Hoeffding Adaptive Tree,Electricity,4.2142,F1: 0.00%,Accuracy: 84.21%


### I.4) Comparisons of the performances

XXX - Comment 

In [24]:
# XXX - delete the datasets (they take up space)

del Hyper100k
del Datasets_capymoa
del Datasets_river

## Part II - Multi_class Classifiers

The multi-class classifiers I found on both River and CapyMOA were : 

- ADWIN Bagging, (capymoa OnlineAdwinBagging)
- Adaptive Random Forest,
- Bagging, 
- Hoeffding Adaptive Tree, 
- Hoeffding Tree, 
- Leveraging Bagging, 
- Naive Bayes, 
- k-Nearest Neighbors, 
- sklearn SGDClassifier


We will compare both libraries on these classifiers.

The datasets we will use are from CapyMOA and are the following : 
 - CovtFD
 - Covtype
 - Sensor

### II.1) Getting the data from CapyMOA 

# CovtFD
CovtFD = datasets.CovtFD()
variables = [str(i) for i in range (1, 105)] #XXX - get the real variables
dataset_to_csv(CovtFD, 'CovtFD.csv', variables)

#Covtype
CovtypeNorm = datasets.CovtypeNorm()
variables = [str(i) for i in range (1, 55)] #XXX - get the real variables
dataset_to_csv(CovtypeNorm, 'CovtypeNorm.csv', variables)

#Sensor
Sensor = datasets.Sensor()
variables = [str(i) for i in range (1, 6)] #XXX - get the real variables
dataset_to_csv(Sensor, 'Sensor.csv', variables)


### II.2) Evaluating with CapyMOA

In [28]:
#Datasets_capymoa = {'CovtFD' : CovtFD, 'CovtypeNorm': CovtypeNorm, 'Sensor': Sensor}
#Datasets_capymoa = {'CovtypeNorm': CovtypeNorm, 'Sensor': Sensor}
#results_capymoa_multiclass = get_performance_capymoa(Models_capymoa, Datasets_capymoa)


In [29]:
#pd.DataFrame(results_capymoa_multiclass)

### II.3) Performances River

In [31]:
#Datasets_river = {'CovtFD' : ('CovtFD.csv', {'converters': {str(i) : float for i in range (1, 105)}}),
#                 'CovtypeNorm' : ('CovtypeNorm.csv', {'converters': {str(i) : float for i in range (1, 55)}}),
#                  'Sensor': ('Sensor.csv', {'converters': {str(i) : float for i in range (1, 6)}}) }

Metrics_river = {'F1' : metrics.F1(), 'Accuracy' : metrics.Accuracy()}

#results_river_multiclass = get_performance_river(Models_river, Datasets_river, Metrics_river)


In [32]:
#pd.DataFrame(results_river_multiclass)

In [33]:
#del CovtFD
#del Covtype
#del Sensor

## Part III - Regression

The regressors I found on both River and CapyMOA were : 
- Passive-Aggressive Regressor
- Stochastic Gradient Tree
- [baseline] Mean predictor
- k-Nearest Neighbors
- Adaptive Random Forest
  
We will compare both libraries on these regressors.

The datasets we will use are from CapyMOA and are the following : 
XXX

### III.1) Getting the data from CapyMOA 

In [35]:
fried_stream = datasets.Fried()
variables = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] #XXX - get the real variables

dataset_to_csv(fried_stream, 'Fried.csv', variables)


In [36]:
bike_stream = datasets.Bike()
variables = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'] #XXX - get the real variables

dataset_to_csv(bike_stream, 'Bike.csv', variables)


### III.2) Evaluating with CapyMOA

In [38]:
Datasets_capymoa = {'Fried' : fried_stream, 'Bike': bike_stream}
Models_capymoa = {'Passive-Aggressive Regressor': PassiveAggressiveRegressor,
         #'Stochastic Gradient Tree': XXX,
         '[baseline] Mean predictor': TargetMean,
         'k-Nearest Neighbors': KNNRegressor,
         'Adaptative Random Forest': AdaptiveRandomForestRegressor}

results_capymoa = get_performance_capymoa(Models_capymoa, Datasets_capymoa)


CapyMOA : Model Passive-Aggressive Regressor and dataset Fried


/opt/anaconda3/lib/python3.12/site-packages/capymoa/evaluation/evaluation.py:348: UserWarning: The learner did not produce a prediction for this instance
  warnings.warn("The learner did not produce a prediction for this instance")


CapyMOA : Model Passive-Aggressive Regressor and dataset Bike


/opt/anaconda3/lib/python3.12/site-packages/capymoa/evaluation/evaluation.py:348: UserWarning: The learner did not produce a prediction for this instance
  warnings.warn("The learner did not produce a prediction for this instance")


CapyMOA : Model [baseline] Mean predictor and dataset Fried
CapyMOA : Model [baseline] Mean predictor and dataset Bike
CapyMOA : Model k-Nearest Neighbors and dataset Fried
CapyMOA : Model k-Nearest Neighbors and dataset Bike
CapyMOA : Model Adaptative Random Forest and dataset Fried
CapyMOA : Model Adaptative Random Forest and dataset Bike


In [39]:
pd.DataFrame(results_capymoa)

,Model,Dataset,Time (seconds),RMSE,MAE,R2
0,Passive-Aggressive Regressor,Fried,9.0246,3.556168,2.786693,0.493842
1,Passive-Aggressive Regressor,Bike,3.9404,103.982219,67.370865,0.671401
2,[baseline] Mean predictor,Fried,0.1871,4.998494,4.065592,0.000000
3,[baseline] Mean predictor,Bike,0.0195,181.395165,131.803362,0.000000
4,k-Nearest Neighbors,Fried,3.2840,2.875860,2.281269,0.668944
5,k-Nearest Neighbors,Bike,1.5413,127.939436,83.791926,0.502545
6,Adaptative Random Forest,Fried,37.5093,2.196962,1.720786,0.806818
7,Adaptative Random Forest,Bike,6.7306,97.432459,65.933862,0.711493


XXX - Comment

### III.3) Evaluating with River

In [41]:
PA_reg_mode1 = preprocessing.StandardScaler() | linear_model.PARegressor(mode=1, C=1.0)
PA_reg_mode2 = preprocessing.StandardScaler() | linear_model.PARegressor(mode=2, C=1.0)
SGT_reg = preprocessing.StandardScaler() | tree.SGTRegressor(max_depth=5)
Mean_reg = preprocessing.StandardScaler() | dummy.StatisticRegressor(stats.Mean())
KNN_reg = preprocessing.StandardScaler() | neighbors.KNNRegressor()
ARF_reg = ARF_reg = (preprocessing.StandardScaler() | forest.ARFRegressor(seed=42))

In [42]:
Datasets_river = {'Fried' : ('Fried.csv', {'converters': {'1': float, '2': float, '3': float, '4': float, '5': float, '6': float, '7': float, '8': float, '9': float, '10': float, 'target':float}})
                  ,'Bike' : ('Bike.csv', {'converters': {'1': float, '2': float, '3': float, '4': float, '5': float, '6': float, '7': float, '8': float, '9': float, '10': float, '11': float, '12': float, 'target':float}})
                 }

Models_river = {'Passive-Aggressive Regressor, mode 1': PA_reg_mode1,
         'Passive-Aggressive Regressor, mode 2': PA_reg_mode2,
         'Stochastic Gradient Tree': SGT_reg,
         '[baseline] Mean predictor': Mean_reg,
         'k-Nearest Neighbors': KNN_reg,
         'Adaptative Random Forest': ARF_reg}

Metrics_river = {'MAE' : metrics.MAE(), 'RMSE' : metrics.RMSE(), 'R2' : metrics.R2()}

Regressors_results_river = get_performance_river(Models_river, Datasets_river, Metrics_river)


River : Model Passive-Aggressive Regressor, mode 1 and dataset Fried
River : Model Passive-Aggressive Regressor, mode 1 and dataset Bike
River : Model Passive-Aggressive Regressor, mode 2 and dataset Fried
River : Model Passive-Aggressive Regressor, mode 2 and dataset Bike
River : Model Stochastic Gradient Tree and dataset Fried
River : Model Stochastic Gradient Tree and dataset Bike
River : Model [baseline] Mean predictor and dataset Fried
River : Model [baseline] Mean predictor and dataset Bike
River : Model k-Nearest Neighbors and dataset Fried
River : Model k-Nearest Neighbors and dataset Bike
River : Model Adaptative Random Forest and dataset Fried
River : Model Adaptative Random Forest and dataset Bike


In [43]:
pd.DataFrame(Regressors_results_river)

,Model,Dataset,Time (seconds),MAE,RMSE,R2
0,"Passive-Aggressive Regressor, mode 1",Fried,0.9010,MAE: 6.035251,RMSE: 7.00572,R2: -0.96549
1,"Passive-Aggressive Regressor, mode 1",Bike,0.4276,MAE: 71.063263,RMSE: 117.753898,R2: 0.578536
2,"Passive-Aggressive Regressor, mode 2",Fried,0.8779,MAE: 10.270834,RMSE: 11.842039,R2: -4.615887
3,"Passive-Aggressive Regressor, mode 2",Bike,0.4107,MAE: 155.881486,RMSE: 227.5805,R2: -0.574272
4,Stochastic Gradient Tree,Fried,4.6792,MAE: 2.596053,RMSE: 3.353492,R2: 0.54964
5,Stochastic Gradient Tree,Bike,0.8476,MAE: 161.555456,RMSE: 234.794335,R2: -0.675657
6,[baseline] Mean predictor,Fried,0.5275,MAE: 4.064122,RMSE: 4.996512,R2: 0.000232
7,[baseline] Mean predictor,Bike,0.2542,MAE: 162.503618,RMSE: 233.203465,R2: -0.653026
8,k-Nearest Neighbors,Fried,70.8528,MAE: 0.,RMSE: 0.,R2: 1.
9,k-Nearest Neighbors,Bike,21.4163,MAE: 15.631639,RMSE: 55.65064,R2: 0.905865
